In [16]:
import math
import datetime
import pandas as pd
import numpy as np
from statistics import median
from scipy.stats import iqr

import time
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [17]:
data_path = '../data/'

In [18]:
items = pd.read_csv('{}items.csv'.format(data_path), sep='|')
infos = pd.read_csv('{}infos.csv'.format(data_path), sep='|')
orders = pd.read_csv('{}orders0206_train.csv'.format(data_path), sep='|')


# Extract additional features on the time-dimension

In [15]:
orderss=orders.head(200)

In [11]:
start = time.clock()
orders['timeb'] = pd.to_datetime(orders["time"])
end = time.clock()
print(end - start)

0.004899200001091231


In [13]:
start = time.clock()
orders['timea'] = orders.apply(lambda row: datetime.datetime.strptime(row['time'],"%Y-%m-%d %H:%M:%S"), axis=1)
end = time.clock()
print(end - start)


0.02091619999919203


In [5]:
orders['time'] = orders.apply(lambda row: datetime.datetime.strptime(row['time'],"%Y-%m-%d %H:%M:%S"), axis=1)

# Month of the year
orders['month'] = orders.apply(lambda row: row['time'].month, axis=1)

# Day of the month
orders['day'] = orders.apply(lambda row: row['time'].day, axis=1)

# Weekday: Starts with 0 = Monday, 6 = Sunday
orders['weekday'] = orders.apply(lambda row: row['time'].weekday(), axis=1)

# Just the date, no time
orders['date'] = orders.apply(lambda row: datetime.datetime(year=2018, month=row['month'], day=row['day']), axis=1)

# Hour of the day
orders['hour'] = orders.apply(lambda row: row['time'].hour, axis=1)

# Calendar week
orders['calendar_week'] = orders.apply(lambda row: row['time'].isocalendar()[1], axis=1)

In [10]:
ex_items = items.copy()
ex_infos = infos.copy()
ex_orders = orders.copy()

# Feature generation: Price deviations

- Diff Sales vs Recommended Price

Item A: 60x für 30€, 30x für 10€

Features:
- Highest Price
- Units Sold with highest price
- Lowest Price
- Units Sold with highest price

In [6]:
joined = pd.merge(left=pd.merge(left=items, right=infos, on='itemID'), right=orders, on='itemID')

In [7]:
joined.head(5)

NameError: name 'train' is not defined

In [8]:
joined['orders_priceDifference'] = joined['salesPrice'] - joined['recommendedRetailPrice']
joined['orders_relPriceDifference'] = joined['orders_priceDifference'] / joined['recommendedRetailPrice']

In [9]:
grouped_orders = orders.groupby("itemID")
minprices = grouped_orders.min()["salesPrice"]
ex_orders = pd.merge(left=orders, right=minprices, on='itemID')
ex_orders.rename(columns={'salesPrice_x':'salesPrice', 'salesPrice_y': 'items_minPrice'} , inplace=True) 

maxprices = grouped_orders.max()["salesPrice"]
ex_orders = pd.merge(left=orders, right=maxprices, on='itemID')
ex_orders.rename(columns={'salesPrice_x':'salesPrice', 'salesPrice_y': 'items_maxPrice'}, inplace=True)

meanprices = grouped_orders.mean()["salesPrice"]
ex_orders = pd.merge(left=orders, right=meanprices, on='itemID')
ex_orders.rename(columns={'salesPrice_x':'salesPrice', 'salesPrice_y': 'items_meanprices'}, inplace=True)


In [37]:
counts = grouped_orders["salesPrice"].value_counts()

ex_items["items_minPriceCount"]= train.apply(lambda row: counts[row["itemID"], row["items_minPrice"]], axis=1)
ex_items["items_maxPriceCount"]= train.apply(lambda row: counts[row["itemID"], row["items_maxPrice"]], axis=1)

# Sales


- number of unites sold
- % of Overall revenue